In [ ]:
from synthesizer.survey.survey import Survey
from synthesizer.emission_models import IntrinsicEmission
from synthesizer.instruments import FilterCollection, UVJ
from synthesizer.instruments import Instrument
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle import BlackHoles, Galaxy
from synthesizer.kernel_functions import Kernel
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric.morphology import Sersic2D

from unyt import kpc, angstrom, Msun, Myr, Mpc
import numpy as np
from astropy.cosmology import Planck18 as cosmo

In [ ]:
def galaxy_loader(gal_index):
    """Load a fake particle Galaxy."""
    # Initialise the parametric Stars object
    param_stars = ParametricStars(
        grid.log10age,
        grid.metallicity,
        sf_hist=SFH.Constant(max_age= 100 * Myr),
        metal_dist=ZDist.DeltaConstant(metallicity=0.01),
        initial_mass=10**10 *Msun,
    )

    # Define the number of stellar particles we want
    n = int(100 * (np.random.rand() + 0.5))

    # Sample the parametric SFZH, producing a particle Stars object
    # we will also pass some keyword arguments for some example attributes
    part_stars = sample_sfhz(
        sfzh=param_stars.sfzh,
        log10ages=param_stars.log10ages,
        log10metallicities=param_stars.log10metallicities,
        nstar=n,
        current_masses=np.full(n, 10**9 / n) * Msun,
        redshift=1,
        coordinates=np.random.normal(0, 0.01, (n, 3)) * Mpc,
        centre=np.zeros(3) * Mpc,
        smoothing_lengths=np.ones(n) * 0.01 * np.random.rand(n) * Mpc,
    )

    # And create the galaxy
    gal = Galaxy(
        stars=part_stars,
        redshift=1,
    )

    return gal

# Get the grid
grid_dir = "../../../tests/test_grid/"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

model = IntrinsicEmission(grid, fesc=0.1)
model.set_per_particle(True)
print(model)

# Get the SPH kernel
sph_kernel = Kernel()
kernel = sph_kernel.get_kernel()

# Get the filters
lam = np.linspace(10**3, 10**5, 1000) * angstrom
webb_filters = FilterCollection(
    filter_codes=[
    f"JWST/NIRCam.{f}"
    for f in ["F090W", "F150W", "F200W", "F277W", "F356W", "F444W"]
],
new_lam=lam,
)
uvj_filters= UVJ(new_lam=lam)

# Instatiate the instruments
webb_inst = Instrument("JWST", filters=webb_filters, resolution=0.1 * kpc)
uvj_inst = Instrument("UVJ", filters=uvj_filters)
instruments = webb_inst + uvj_inst 

print(instruments)

In [ ]:
survey = Survey(galaxy_loader, model, instruments, 10, nthreads=8)
survey.load_galaxies()
survey.get_spectra(cosmo=cosmo)
survey.get_lines(line_ids=grid.available_lines)
survey.get_photometry_luminosities()
survey.get_photometry_fluxes()
survey.get_images_luminosity(fov=50 * kpc, kernel=kernel)
survey.get_images_flux(fov=50* kpc, kernel=kernel)
survey.write("output.hdf5")


In [ ]:
survey.galaxies[0].stars.particle_photo_lnu